In [139]:
import pandas as pd
import numpy as np

In [140]:
data = pd.read_csv("attribution_allocation_student_data.csv")

In [141]:
data_T = data.loc[data.convert_TF==True]

In [142]:
first = []
last = []
middle = []
for row in data_T.iterrows():
    interactions = [row[1]['touch1'],row[1]['touch2'],row[1]['touch3'],row[1]['touch4'],row[1]['touch5']]
    interactions = [i for i in interactions if i == i]
    first.append(interactions[0])
    last.append(interactions[-1])
    if len(interactions)> 2:
        middle.append(interactions[1:-1])
    else:
        middle.append([])

In [143]:
data_T['first'] = first
data_T['last'] = last
data_T['middle'] = middle

C:\Users\xsc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\xsc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\xsc\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

In [144]:
data_T.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13779 entries, 0 to 68117
Data columns (total 10 columns):
convert_TF    13779 non-null bool
touch1        13779 non-null object
touch2        12143 non-null object
touch3        9181 non-null object
touch4        3837 non-null object
touch5        1259 non-null object
tier          13779 non-null int64
first         13779 non-null object
last          13779 non-null object
middle        13779 non-null object
dtypes: bool(1), int64(1), object(8)
memory usage: 1.1+ MB


In [145]:
interactions = ["email","social","display","paid_search","referral","organic_search","direct"] 

In [146]:
#####Last Non-direct Interaction & First Interaction models
l = {}
l1 = {}
l2 = {}
l3 = {}
f = {}
f1 = {}
f2 = {}
f3 = {}
for i in interactions:
    l[i] = data_T['last'][data_T['last']==i].count()
    l1[i] = data_T['last'][(data_T['last']==i) & (data_T['tier']==1)].count()
    l2[i] = data_T['last'][(data_T['last']==i) & (data_T['tier']==2)].count()
    l3[i] = data_T['last'][(data_T['last']==i) & (data_T['tier']==3)].count()
    f[i] = data_T['first'][data_T['first']==i].count()
    f1[i] = data_T['first'][(data_T['first']==i) & (data_T['tier']==1)].count()
    f2[i] = data_T['first'][(data_T['first']==i) & (data_T['tier']==2)].count()
    f3[i] = data_T['first'][(data_T['first']==i) & (data_T['tier']==3)].count()
    
l['social'] = l['social']+1
l['referral'] = l['referral']+3
l1['referral'] = l1['referral']+2
l3['referral'] = l3['referral']+1
l3['social'] = l3['social']+1
l['direct'] = 0
l1['direct'] = 0
l2['direct'] = 0
l3['direct'] = 0

In [147]:
#####Position-based model
p = {}
p1 = {}
p2 = {}
p3 = {}
for i in interactions:
    p[i] = 0
    p1[i] = 0
    p2[i] = 0
    p3[i] = 0
for row in data_T.iterrows():
    first = row[1]['first']
    last = row[1]['last']
    middle = row[1]['middle']
    tier = row[1]['tier']
    if tier == 1:   
        if len(middle)==0:
            p[first] = p[first]+0.5
            p1[first] = p1[first]+0.5
            p[last] = p[last]+0.5
            p1[last] = p1[last]+0.5
        else:
            p[first] = p[first]+0.4
            p1[first] = p1[first]+0.4
            p[last] = p[last]+0.4
            p1[last] = p1[last]+0.4
            for touch in middle:
                p[touch] = p[touch]+0.2/len(middle)
                p1[touch] = p1[touch]+0.2/len(middle)
           
    elif tier == 2:
        if len(middle)==0:
            p[first] = p[first]+0.5
            p2[first] = p2[first]+0.5
            p[last] = p[last]+0.5
            p2[last] = p2[last]+0.5
        else:
            p[first] = p[first]+0.4
            p2[first] = p2[first]+0.4
            p[last] = p[last]+0.4
            p2[last] = p2[last]+0.4
            for touch in middle:
                p[touch] = p[touch]+0.2/len(middle)
                p2[touch] = p2[touch]+0.2/len(middle)
           
    else:
        if len(middle)==0:
            p[first] = p[first]+0.5
            p3[first] = p3[first]+0.5
            p[last] = p[last]+0.5
            p3[last] = p3[last]+0.5
        else:
            p[first] = p[first]+0.4
            p3[first] = p3[first]+0.4
            p[last] = p[last]+0.4
            p3[last] = p3[last]+0.4
            for touch in middle:
                p[touch] = p[touch]+0.2/len(middle)
                p3[touch] = p3[touch]+0.2/len(middle)

In [148]:
cac_summary = {}
for i in interactions[0:5]:
    cac = {}
    cac['L_margin_tier1'] = 1000/l1[i]
    cac['L_margin_tier2'] = 1000/(l2[i]-l1[i])
    cac['L_margin_tier3'] = 1000/(l3[i]-l2[i])
    cac['L_overall'] = 6000/l[i]
    cac['F_margin_tier1'] = 1000/f1[i]
    cac['F_margin_tier2'] = 1000/(f2[i]-f1[i])
    cac['F_margin_tier3'] = 1000/(f3[i]-f2[i])
    cac['F_overall'] = 6000/f[i]
    cac['P_margin_tier1'] = 1000/p1[i]
    cac['P_margin_tier2'] = 1000/(p2[i]-p1[i])
    cac['P_margin_tier3'] = 1000/(p3[i]-p2[i])
    cac['P_overall'] = 6000/p[i]
    cac_summary[i] = cac
    

In [149]:
cac_summary = pd.DataFrame(cac_summary)

In [150]:
cac_summary

,email,social,display,paid_search,referral
F_margin_tier1,4.065041,2.123142,2.066116,7.812500,0.669792
F_margin_tier2,5.434783,3.058104,3.984064,8.403361,0.846740
F_margin_tier3,6.289308,5.263158,9.523810,13.333333,1.564945
F_overall,4.743083,2.658396,2.914036,8.608321,0.802139
L_margin_tier1,3.344482,1.607717,1.883239,5.555556,0.841043
L_margin_tier2,4.291845,2.237136,3.412969,6.250000,1.069519
L_margin_tier3,6.493506,5.319149,7.407407,6.944444,1.828154
L_overall,3.955175,2.035278,2.592913,5.976096,1.002674
P_margin_tier1,3.869969,1.950332,2.051563,7.067138,0.704275
P_margin_tier2,5.175091,2.713459,3.807590,7.747934,0.898015
